## Deliverable 2. Create a Customer Travel Destinations Map.

In [66]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [67]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather-Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Qaanaaq,GL,77.4840,-69.3632,19.26,59,44,9.95
1,1,Surt,LY,31.2089,16.5887,81.23,59,0,11.70
2,2,Vaini,TO,-21.2000,-175.2000,73.72,78,77,21.99
3,3,Hithadhoo,MV,-0.6000,73.0833,81.73,74,91,7.90
4,4,Touros,BR,-5.1989,-35.4608,76.62,78,41,17.31


In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Surt,LY,31.2089,16.5887,81.23,59,0,11.70
3,3,Hithadhoo,MV,-0.6000,73.0833,81.73,74,91,7.90
11,11,Twentynine Palms,US,34.1356,-116.0542,89.98,11,0,9.95
14,14,Jacareacanga,BR,-6.2222,-57.7528,80.04,66,92,1.99
17,17,Le Robert,MQ,14.6775,-60.9392,80.65,83,20,3.44


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID       166
City          166
Country       166
Lat           166
Lng           166
Max Temp      166
Humidity      166
Cloudiness    166
Wind Speed    166
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Surt,LY,31.2089,16.5887,81.23,59,0,11.70
3,3,Hithadhoo,MV,-0.6000,73.0833,81.73,74,91,7.90
11,11,Twentynine Palms,US,34.1356,-116.0542,89.98,11,0,9.95
14,14,Jacareacanga,BR,-6.2222,-57.7528,80.04,66,92,1.99
17,17,Le Robert,MQ,14.6775,-60.9392,80.65,83,20,3.44


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Surt,LY,81.23,31.2089,16.5887,
3,Hithadhoo,MV,81.73,-0.6000,73.0833,
11,Twentynine Palms,US,89.98,34.1356,-116.0542,
14,Jacareacanga,BR,80.04,-6.2222,-57.7528,
17,Le Robert,MQ,80.65,14.6775,-60.9392,
19,Road Town,VG,81.59,18.4167,-64.6167,
29,Eufaula,US,78.33,31.8913,-85.1455,
36,Kapaa,US,87.78,22.0752,-159.3190,
38,Hasaki,JP,84.29,35.7333,140.8333,
41,Butaritari,KI,82.67,3.0707,172.7902,


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except(IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Surt,LY,81.23,31.2089,16.5887,فندق المدينة
3,Hithadhoo,MV,81.73,-0.6000,73.0833,Scoop Guest House
11,Twentynine Palms,US,89.98,34.1356,-116.0542,Holiday Inn Express & Suites Twentynine Palms-...
14,Jacareacanga,BR,80.04,-6.2222,-57.7528,Casa do Ian😎
17,Le Robert,MQ,80.65,14.6775,-60.9392,HARMONY HEBERGEMENT


In [48]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Surt,LY,81.23,31.2089,16.5887,فندق المدينة
3,Hithadhoo,MV,81.73,-0.6000,73.0833,Scoop Guest House
11,Twentynine Palms,US,89.98,34.1356,-116.0542,Holiday Inn Express & Suites Twentynine Palms-...
14,Jacareacanga,BR,80.04,-6.2222,-57.7528,Casa do Ian😎
17,Le Robert,MQ,80.65,14.6775,-60.9392,HARMONY HEBERGEMENT


In [49]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))